# Test - Data Availability
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Apr 26, 2022

In [1]:
import getpass
import pandas as pd
import pyodbc

### Create a connection to MSSQL PROD:

In [2]:
# connect to SQL Server.
SERVER = '52.44.171.130' 
DATABASE = 'datascience' 
USERNAME = 'dsAdminWrite' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

InterfaceError: ('28000', "[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user 'dsAdminWrite'. (18456) (SQLDriverConnect)")

### Now we can run our stored proc to get retention training and inference datasets:

In [14]:
lkupclientid = 51
dbname = 'stlrMILB'

cursor = CNXN.cursor()

storedProc = (
    f"""Exec {dbname}.[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

print(f"TOTAL ROWS: {df.shape[0]}")
df.head()

TOTAL ROWS: 818


,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,...,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,51,277339362,7643347,2017,Full Season,36.0,2017-07-03,1.000000,4,TRUE,...,1,None,None,None,None,0.0,0,75,None,0
1,51,277340000,6665140,2017,Package,114.0,2017-07-15,0.250000,163,TRUE,...,1,None,None,None,None,0.0,0,75,None,1
2,51,277340018,4001332,2017,Package,85.5,2017-08-20,1.000000,190,TRUE,...,1,None,None,None,None,0.0,0,75,None,0
3,51,277340215,5045782,2017,Package,142.5,2017-09-03,1.000000,143,TRUE,...,1,None,None,None,None,0.0,0,75,None,0
4,51,277340695,4115774,2017,Full Season,1632.0,2017-08-24,0.716418,112,TRUE,...,1,None,None,None,None,0.0,0,75,None,1


### Let's check how much data we have for each season:

In [15]:
df["year"].value_counts()

2021    249
2017    209
2018    201
2019    159
Name: year, dtype: int64

### Done